In [42]:
import requests
import math

api_key = 'YOUR_API'

def get_elevation(lat, lon, api_key):
    url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={lat},{lon}&key={api_key}"
    response = requests.get(url)
    result = response.json()
    if result['status'] == 'OK':
        return result['results'][0]['elevation']
    else:
        raise Exception(f"API Error: {result['status']}")

def haversine(lat1, lon1, lat2, lon2):
    R = 6371e3  # Radius of the Earth in meters
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c  # Distance in meters

def calculate_slope(elev1, elev2, distance):
    return (elev2 - elev1) / distance * 100  # Slope as a percentage

def get_surrounding_points(lat, lon, distance_meters, num_points=72):
    points = []
    angle_increment = 360 / num_points
    earth_radius = 6371e3  # Radius of Earth in meters

    for i in range(num_points):
        angle = math.radians(i * angle_increment)
        delta_lat = distance_meters / earth_radius * (180 / math.pi)
        delta_lon = delta_lat / math.cos(math.radians(lat))

        lat2 = lat + delta_lat * math.cos(angle)
        lon2 = lon + delta_lon * math.sin(angle)

        points.append((lat2, lon2, i * angle_increment))

    return points

def find_steepest_descent(lat, lon, api_key, distance_meters=10, num_points=72):
    # Get elevation for the center point
    elev_center = get_elevation(lat, lon, api_key)

    # Get surrounding points in a circle around the center point
    points = get_surrounding_points(lat, lon, distance_meters, num_points)

    steepest_slope = None
    steepest_bearing = None

    for lat2, lon2, bearing in points:
        # Get elevation of the surrounding point
        elev2 = get_elevation(lat2, lon2, api_key)

        # Calculate horizontal distance between points
        distance = haversine(lat, lon, lat2, lon2)

        # Calculate slope
        slope = calculate_slope(elev_center, elev2, distance)

        if steepest_slope is None or slope < steepest_slope:
            steepest_slope = slope
            steepest_bearing = bearing

    return elev_center, steepest_slope, steepest_bearing

# Example usage
lat, lon = 34.902871, 139.097548# Example location: Denver, CO

elevation, steepest_slope, bearing = find_steepest_descent(lat, lon, api_key)

print(f"Steepest Slope: {steepest_slope:.2f}%")
print(f"Bearing (direction): {bearing:.2f}°")

Steepest Slope: -56.66%
Bearing (direction): 120.00°


In [35]:
import pandas as pd

points = pd.read_csv('CA_locations_2000.csv')
points


,Index,Longitude,Latitude,have_plant
0,0,-120.109391,36.177598,1
1,1,-116.327979,33.265240,1
2,2,-118.304671,34.707428,1
3,3,-118.274109,34.708588,1
4,4,-120.115552,36.175836,1
...,...,...,...,...
2310,2310,-118.896501,36.718088,0
2311,2311,-119.133723,36.125849,0
2312,2312,-122.722559,39.423085,0
2313,2313,-118.025948,34.326249,0


In [37]:
points[0:5]

,Index,Longitude,Latitude,have_plant
0,0,-120.109391,36.177598,1
1,1,-116.327979,33.265240,1
2,2,-118.304671,34.707428,1
3,3,-118.274109,34.708588,1
4,4,-120.115552,36.175836,1


In [54]:
mat = []

conn = sqlite3.connect('download_tracker.sql')
c = conn.cursor()

for x in points[1876:].iloc:
    I, LON,LAT,PLANT = x
    I = int(I)
    PLANT = int(PLANT)
    #print(I,LON,LAT,PLANT)
    elevation, steepest_slope, bearing = find_steepest_descent(LAT, LON, api_key)
    print([I, LON, LAT, elevation, steepest_slope, bearing, PLANT])
    mat.append([I, LON, LAT, elevation, steepest_slope, bearing, PLANT])
    query = f'INSERT INTO POINTS2(I,Longitude,Latitude,elevation,slope,slope_bearing,have_plant) VALUES ({I},{LON},{LAT},{elevation},{steepest_slope},{bearing},{PLANT})'
    c.execute(query)
    conn.commit()

    #mat.append()

[1876, -121.79545225972907, 38.8633191298322, 10.65197944641113, -2.877216915143396, 40.0, 0]
[1877, -116.7176009724955, 35.4483560538792, 975.4005737304688, -1.961669575078682, 220.0, 0]
[1878, -116.0491573787384, 35.99831250842546, 1237.105590820312, -16.552732095775387, 105.0, 0]
[1879, -121.73823759689064, 40.18007016409181, 616.2400512695312, -24.749141086819034, 145.0, 0]
[1880, -115.86329486083308, 34.658976005673445, 617.63818359375, -29.198602421805365, 230.0, 0]
[1881, -118.60593267206704, 36.87853342415482, 3325.30224609375, -41.75293127722501, 345.0, 0]
[1882, -117.48359384115348, 33.61391129584339, 703.0413208007812, -35.89415853939792, 245.0, 0]
[1883, -123.45055955511812, 41.67497602398435, 528.591064453125, -20.43395611351023, 145.0, 0]
[1884, -120.83182450623104, 41.47067902813835, 1379.886108398438, -7.3632809192421265, 165.0, 0]
[1885, -120.27084960419756, 38.82257111365453, 2075.47900390625, -5.346679905777721, 345.0, 0]
[1886, -121.27239291925848, 37.32997481938175

In [61]:
conn = sqlite3.connect('download_tracker.sql')
geo = pd.read_sql_query(f'SELECT * from points2',conn)

geo = geo.drop_duplicates()

geo.to_csv('points2.csv')

In [49]:
#import sqlite3

#mat = pd.DataFrame(mat)

#conn = sqlite3.connect('download_tracker.sql')

#mat.columns = ['I','Longitude','Latitude','elevation','slope','slope_bearing','have_plant']

#mat.to_sql('POINTS2', conn, index=False, if_exists='replace')

5

In [55]:
mat

[[1876,
  -121.79545225972907,
  38.8633191298322,
  10.65197944641113,
  -2.877216915143396,
  40.0,
  0],
 [1877,
  -116.7176009724955,
  35.4483560538792,
  975.4005737304688,
  -1.961669575078682,
  220.0,
  0],
 [1878,
  -116.0491573787384,
  35.99831250842546,
  1237.105590820312,
  -16.552732095775387,
  105.0,
  0],
 [1879,
  -121.73823759689064,
  40.18007016409181,
  616.2400512695312,
  -24.749141086819034,
  145.0,
  0],
 [1880,
  -115.86329486083308,
  34.658976005673445,
  617.63818359375,
  -29.198602421805365,
  230.0,
  0],
 [1881,
  -118.60593267206704,
  36.87853342415482,
  3325.30224609375,
  -41.75293127722501,
  345.0,
  0],
 [1882,
  -117.48359384115348,
  33.61391129584339,
  703.0413208007812,
  -35.89415853939792,
  245.0,
  0],
 [1883,
  -123.45055955511812,
  41.67497602398435,
  528.591064453125,
  -20.43395611351023,
  145.0,
  0],
 [1884,
  -120.83182450623104,
  41.47067902813835,
  1379.886108398438,
  -7.3632809192421265,
  165.0,
  0],
 [1885,
  -120